In [7]:
-- Prolog term
data Term = V String      -- logic variable.       String 대문자로 시작
          | T String [Term] -- atom when empty list. String 소문자로 시작
      deriving (Eq,Ord,Show)

일치화 규칙

$\displaystyle
[\texttt{a}=\texttt{a},~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[\texttt{f}(t_1,\ldots,t_n)=\texttt{f}(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [8]:
occurs :: String -> Term -> Bool
occurs x (V x') = x==x'
occurs x (T _ ts) = any (occurs x) ts

In [9]:
subst (x,t) u@(V y)  | x == y    = t
                     | otherwise = u
subst (x,t) (T f us) = T f [subst (x,t) u | u<-us]

In [10]:
int = T "int" []
subst ("x",T"f"[int,int]) (T"f"[V"x",V"x"])

T "f" [T "f" [T "int" [],T "int" []],T "f" [T "int" [],T "int" []]]

In [14]:
type Eqn = (Term,Term)     -- t1 = t2
type Subst = [(String,Term)]

-- 일치화 알고리듬을 위 규칙에 따라 작성
unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((T f ts, T g us): es, s) | f==g  = unif (zip ts us ++ es, s)
unif ((V x   , V x'  ): es, s) | x==x' = unif (es, s)
unif ((V x   , t     ): es, s) | not(x `occurs` t) = unif (es', (x,t):s')
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
        s' = [(x',subst (x,t) t') | (x',t')<-s]
unif ((t     , V x   ): es, s) = unif ((V x,t):es, s)
unif (es, s) = (es, s)